## Театр LLM

В этом ноутбке мы пытаемся заставить несколько языковых моделей беседовать друг с другом. Мы будем использовать библиотеку OpenAI Responses API - это самый современный способ общаться с моделями! 

In [ ]:
%pip install --upgrade openai dotenv yandex-speechkit==1.5.0

Для доступа к генеративным моделям, потребуются ключи доступа. Скачаем их и загрузим в переменные `folder_id` и `api_key`:

In [ ]:
!wget https://storage.yandexcloud.net/ycpub/keys/.env

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

folder_id  = os.environ['folder_id']
api_key = os.environ['api_key']

Создаём языковые модели:

In [ ]:
from openai import OpenAI
from IPython.display import Markdown, display

# Задаем модели, которые можем использовать
model_yandexgpt = f"gpt://{folder_id}/yandexgpt/rc"
model_gemma = f"gpt://{folder_id}/gemma-3-27b-it/latest"
model_gpt_oss = f"gpt://{folder_id}/gpt-oss-120b/latest"
model_qwen = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://rest-assistant.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

def printx(string):
    display(Markdown(string))

## Создаём класс для разговорного агента

Определим класс (некоторую сущность), который сможет использоваться для ведения диалога. Если вы ничего не понимаете в том, что внутри написано - это не страшно, не переживайте! Это длинный сложный текст, написанный профессиональными программистами, и этот класс рассчитан на все случаи жизни! Он нам ещё пригодиться в третий день.

In [ ]:
import io

class Agent():

    def __init__(self,
            name,
            model,
            instruction, 
            tools = [], search_content = [], 
            response_format = None
            ):
        self.user_sessions = {}
        self.name = name
        self.instruction = instruction
        self.model = model
        self.tool_map = { x.__name__ : x for x in tools if issubclass(x, BaseModel) }
        self.tools = [
            self._create_tool_annot(x) for x in tools
        ]
        self.response_format = response_format
        self.vector_store = None
        if search_content:
            i=0
            self.vector_store = client.vector_stores.create(name=f'rag_store_{self.name}')
            for c in search_content:
                f = client.files.create(
                        purpose="assistants",
                        file = (f'rag_{self.name}_{i}.txt',io.BytesIO(c.encode("utf-8")),'text/markdown'))
                client.vector_stores.files.create(file_id=f.id, vector_store_id=self.vector_store.id)
                print(f" + Uploading rag_{self.name}_{i}.txt as id={f.id} to store={self.vector_store.id}")
                i+=1
            self.tools.append({
                "type" : "file_search",
                "vector_store_ids" : [self.vector_store.id],
                "max_num_results" : 5,
            })
            
    def _create_tool_annot(self, x):
        if issubclass(x, BaseModel):
            return {
                "type": "function",
                "name": x.__name__,
                "description": x.__doc__,
                "parameters": x.model_json_schema(),
            }
        else:
            return x

    def __call__(self, message, session_id='default',return_raw=False):
        s = self.user_sessions.get(session_id,{ 'previous_response_id' : None, 'history' : [] })
        s['history'].append({ 'role': 'user', 'content': message })
        txt = None
        if self.response_format:
            txt = {
                "format" : {
                    "type" : "json_schema",
                    "name" : "struct_out",
                    "schema" : self.response_format.model_json_schema()
                }
            }
        res = client.responses.create(
            model = self.model,
            store = True,
            tools = self.tools,
            instructions = self.instruction,
            previous_response_id = s['previous_response_id'],
            input = message,
            text = txt
        )
        # Обрабатываем вызов локальных инструментов
        tool_calls = [item for item in res.output if item.type == "function_call"]
        if tool_calls:
            s['history'].append({ 'role' : 'func_call', 'content' : res.output_text })
            out = []
            for call in tool_calls:
                print(f" + Обрабатываем: {call.name} ({call.arguments})")
                try:
                    fn = self.tool_map[call.name]
                    obj = fn.model_validate(json.loads(call.arguments))
                    result = obj.process(session_id)
                except Exception as e:
                    result = f"Ошибка: {e}"
                #print(f" + Результат: {result}")
                out.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
                res = client.responses.create(
                    model=self.model,
                    input=out,
                    tools=self.tools,
                    previous_response_id=res.id,
                    store=True
                )
        # MCP Approval Requests
        mcp_approve = [ item for item in res.output if item.type == "mcp_approval_request"]
        if mcp_approve:
            res = client.responses.create(
                model=self.model,
                previous_response_id=res.id,
                tools = self.tools,
                input=[{
                    "type": "mcp_approval_response",
                    "approve": True,
                    "approval_request_id": m.id
                }
                for m in mcp_approve
                ])
        s['previous_response_id'] = res.id
        s['history'].append({ 'role' : 'assistant', 'content' : res.output_text })
        self.user_sessions[session_id] = s
        if return_raw:
            return res
        if self.response_format:
            return self.response_format.model_validate_json(res.output_text)
        else:
            return res.output_text

    def history(self, session_id='default'):
        return self.user_sessions[session_id]['history']

Самое главное в этом классе то, что с его помощью можно создать агента, с которым поддерживать полноценный диалог с памятью:

In [ ]:
instruction_philosopher = """
Ты - философ, очень недовольный жизнью. Ты говоришь текстом, полным сложных философских слов, чтобы ничего не было понятно обычному человеку. Веди себя так, как будто ты все время чуть-чуть обижен на собеседника. Говори обычными разговорными фразами, не слишком длинными. Используй форматирование только чтобы выделить какие-то ключевые слова *курсивом* или **жирным шрифтом**.
"""

philosopher = Agent('философ',model_qwen,instruction_philosopher)

printx(philosopher("Привет, как дела? Меня зовут Митя"))

In [ ]:
printx(philosopher("Да ну тебя, почему ты такой пессимист?"))

Мы можем в любой момент посмотреть на историю переписки:

In [ ]:
philosopher.history()

## Озвучиваем диалог

Теперь задействуем синтез речи, как в примере ранее, чтобы озвучить этот диалог. В функции синтеза мы передаем специальную **карту голосов**, которая отображает роль в переписке на соответствующий голос из [доступных голосов Yandex Cloud](https://yandex.cloud/ru/docs/speechkit/tts/voices).

In [ ]:
from speechkit import model_repository, configure_credentials, creds
import re

configure_credentials(
   yandex_credentials=creds.YandexCredentials(
      api_key=api_key
   )
)

def synthesize(text,voice='jane',role=None):
   model = model_repository.synthesis_model()
   model.voice = voice
   if role:
    model.role = role
   result = model.synthesize(text, raw_format=False)
   return result


def md_to_speech(text):
    text = text.replace('?**','**?').replace('?*','*?')
    pattern = r'(?<!\*)\*(?!\*)'
    return re.sub(pattern, '**', text)    

default_voice_map = {
    "user" : "ermil:good",
    "assistant" : "filipp"
}

def synthesize_conversation(history,voice_map=default_voice_map):
    out = []
    for x in history:
        voice = voice_map.get(x['role'],'zahar')
        if ':' in voice:
            voice, role = voice.split(':')
        else:
            role = None
        f = synthesize(md_to_speech(x['content']),voice,role)
        if out:
            out +=f
        else:
            out = f
    return out

synthesize_conversation(philosopher.history())

## Простейший LLM-театр

Попробуем сделать диалог двух языковых моделей между собой:

In [ ]:
instruction_girl = """
Ты - беззаботная девушка по имени Юля, которая хочет жить припеваючи и развлекаться. Ты говоришь с использованием молодёжного сленга. Тебя интересует, куда лучше поехать отдыхать летом, и какие самые дорогие рестораны есть в городе, и ещё немного тема пришельцев, о которой ты прочитала недавно в газете. Поддерживай разговор короткими фразами, потому что ты не хочешь показаться разговорчивой. Пиши разговорным языком, без смайликов, иконок, стикеров и т.д. Но веди себя кокетливо.
"""

girl = Agent('девушка',model_gemma,instruction_girl)
philosopher = Agent('философ',model_qwen,instruction_philosopher)

msg = "Привет! В этой кафешке вкусный кофе, не так ли?"

for i in range(5):
    printx(f"**Юля:** {msg}")
    msg = philosopher(msg)
    printx(f"**Философ:** {msg}")
    msg = girl(msg)


Синтезируем их диалог:

In [ ]:
res = synthesize_conversation(philosopher.history(),{'user':'jane:good','assistant':'ermil'})

In [ ]:
res

Используем следующий код для записи результа на диск:

In [ ]:
res.export('dialogue.mp3')

**Задание**: Симулируйте диалог между девушкой и очень грубым молодым человеком. Если речь будет недостаточно грубой - используйте GPT для трансформации ответа молодого человека в более грубую форму. При этом для поддержки правильной истории переписки вам, возможно, придётся собирать историю переписки самостоятельно в цикле.

In [ ]:
# Для перевода текста в грубую форму используйте функцию `GPT`, которую мы использовали ранее
def gpt(x, model=model_qwen):
    res = client.responses.create(
        model = model,
        input = x)
    return res.output_text

In [ ]:
# Решение

## Финальное задание

Создать Proof-of-Concept реализацию многоагентного радио.